## finetune

In [1]:
%pip install datasets accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset
dataset = load_dataset('Romjiik/Russian_bank_reviews')



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("igorktech/rugpt3-joker-150k")
tokenizer.pad_token = tokenizer.eos_token


tokenizer_config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.74M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [4]:
tokenizer.eos_token_id

50257

In [5]:


def tokenize_function(examples):
    return tokenizer(examples['review'], padding=True, truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Model
model = AutoModelForCausalLM.from_pretrained("igorktech/rugpt3-joker-150k")


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=0.2,
    per_device_train_batch_size=2,
    save_steps=1000,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
)

# Train the model



Map:   0%|          | 0/12392 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [6]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


KeyboardInterrupt: 

In [7]:
trainer.save_model("joker_model")

## to onnx

In [103]:
from optimum.onnxruntime import ORTModelForCausalLM
from transformers import AutoTokenizer

model_checkpoint = "joker_model"
save_directory = "onnx/"

# Load a model from transformers and export it to ONNX
ort_model = ORTModelForCausalLM.from_pretrained(
    model_checkpoint,
    use_cache=False,
    use_io_binding=False,
    export=True,
  )


Framework not specified. Using pt to export to ONNX.
The task `text-generation` was manually specified, and past key values will not be reused in the decoding. if needed, please pass `--task text-generation-with-past` to export using the past key values.
Using the export variant default. Available variants are:
    - default: The default ONNX variant.
Using framework PyTorch: 2.1.0+cu121
Overriding 2 configuration item(s)
	- use_cache -> False
	- pad_token_id -> 0
/usr/local/lib/python3.10/dist-packages/transformers/models/gpt2/modeling_gpt2.py:801: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if batch_size <= 0:
Post-processing the exported models...
Deduplicating shared (tied) weights...
Found different candidate ONNX initializers (likely duplicate) for the tied wei

In [49]:
ort_model.save_pretrained(save_directory)

In [33]:
from tokenizers import Tokenizer
tkn = Tokenizer.from_pretrained("igorktech/rugpt3-joker-150k")

In [34]:
tkn.save("tokenizer.bin")

In [8]:
inputs = tkn.encode("кек прикольный прикол")

In [21]:
tokenizer("to wo")

{'input_ids': [11874, 544, 83], 'attention_mask': [1, 1, 1]}

In [10]:
!zip onnx.zip ./onnx/*

  adding: onnx/added_tokens.json (stored 0%)
  adding: onnx/config.json (deflated 51%)
  adding: onnx/merges.txt (deflated 76%)
  adding: onnx/model.onnx (deflated 8%)
  adding: onnx/special_tokens_map.json (deflated 74%)
  adding: onnx/tokenizer_config.json (deflated 55%)
  adding: onnx/tokenizer.json (deflated 80%)
  adding: onnx/vocab.json (deflated 74%)


In [ ]:
dir(model)

In [11]:
pt_inputs = tokenizer("заходит в бар ули", return_tensors="pt")

In [13]:
pt_inputs["input_ids"]

tensor([[ 431, 1454,  282, 4316, 2096]])

In [24]:
type(model)

transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel

In [28]:
%pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 42.1 MB/s eta 0:00:00


In [8]:
type(model)

transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel

In [19]:
import torch

class ModelPath(torch.nn.Module):
  def __init__(self, model):
    super().__init__()
    self.model = model

  def forward(self, ids, attn):
    return self.model(input_ids=ids, attention_mask = attn, return_dict=False)

model_patch = ModelPath(model)


In [20]:
%%time
args = (
    pt_inputs["input_ids"],
    pt_inputs["attention_mask"]
    )
out = model_patch(*args)

CPU times: user 176 ms, sys: 0 ns, total: 176 ms
Wall time: 198 ms


In [23]:
pt_inputs

{'input_ids': tensor([[ 431, 1454,  282, 4316, 2096]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [25]:
torch.argmax(out[0][0][-1])

tensor(6171)

In [27]:
tokenizer.decode([6171])

'тка'

In [99]:
out[1][0][0].shape

torch.Size([1, 12, 4, 64])

In [30]:
torch.onnx.export(
    model_patch,
    args,
    "decoder.onnx",
    input_names=["ids", "attention_mask"],
    output_names=["logits"],
    dynamic_axes={
        "ids": {1: "seq_len"},
        "attention_mask": {1: "seq_len"}
    },
    use_fp16=True
 )

## inference onnx

In [31]:
%pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.2 MB/s eta 0:00:00


In [35]:
from tokenizers import Tokenizer
from onnxruntime import InferenceSession

tokenizer = Tokenizer.from_file("tokenizer.bin")
session = InferenceSession("decoder.onnx")

In [45]:
# ONNX Runtime expects NumPy arrays as input
inputs = tokenizer.encode("заходит в бар ули")
inputs

Encoding(num_tokens=5, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [46]:
import numpy as np
input_feed = {
    "ids": np.array([inputs.ids]),
    "attention_mask": np.array([inputs.attention_mask])
    }

In [47]:
%%time
outputs = session.run(
    output_names=["logits"],
    input_feed=input_feed
    )

CPU times: user 65.5 ms, sys: 1.54 ms, total: 67.1 ms
Wall time: 74.4 ms


In [39]:
outputs[0][0][-1].shape

(50264,)

In [40]:
out_ids = np.argmax(outputs[0][0][-1])

In [58]:
tkn.decode([out_ids])

'тка'

In [48]:
out[0][0].shape

torch.Size([5, 50264])

In [49]:
outputs[0][0].shape

(5, 50264)

In [59]:
np.allclose(out[0][0][-1].detach().numpy(), outputs[0][0][-1], atol=1e-01)

False

In [60]:
!ls -lah

total 634M
drwxr-xr-x 1 root root 4.0K Feb 10 13:35 .
drwxr-xr-x 1 root root 4.0K Feb 10 13:14 ..
drwxr-xr-x 4 root root 4.0K Feb  8 14:20 .config
-rw-r--r-- 1 root root 630M Feb 10 13:33 decoder.onnx
drwxr-xr-x 2 root root 4.0K Feb 10 13:18 joker_model
drwxr-xr-x 3 root root 4.0K Feb 10 13:17 output
drwxr-xr-x 1 root root 4.0K Feb  8 14:21 sample_data
-rw-r--r-- 1 root root 3.6M Feb 10 13:35 tokenizer.bin


In [62]:
dir(tokenizer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'add_special_tokens',
 'add_tokens',
 'decode',
 'decode_batch',
 'decoder',
 'enable_padding',
 'enable_truncation',
 'encode',
 'encode_batch',
 'encode_special_tokens',
 'from_buffer',
 'from_file',
 'from_pretrained',
 'from_str',
 'get_added_tokens_decoder',
 'get_vocab',
 'get_vocab_size',
 'id_to_token',
 'model',
 'no_padding',
 'no_truncation',
 'normalizer',
 'num_special_tokens_to_add',
 'padding',
 'post_process',
 'post_processor',
 'pre_tokenizer',
 'save',
 'to_str',
 'token_to_id',
 'train',
 'train_from_iterator',
 'truncation']